In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

from tensorflow.keras.losses import CategoricalCrossentropy,SparseCategoricalCrossentropy
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import callbacks

from keras import backend as K

from imutils import paths

from tensorflow.keras.utils import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.utils import np_utils

import cv2

import os
import imutils
import glob

In [ ]:
# load in image
# rescale to 48*48
# generate more data with shift rotate grey scale?
# d = Image.open('.././data/images/train/angry/0.jpg')

In [26]:
path= '.././data/2023-12-28/ukdata/utkface_aligned_cropped/UTKFace'
ages = []
imgs_list = []

In [27]:
file_list = glob.glob(path+'/*.jpg')

In [28]:
# 1-10: Child
# 11-20: Teen
# 21-30: Young Adult
# 31-40: Middle Ages Adult
# 41-50: Senior Adult
# 51-70: Old Adult
# 71-80: Elderly
# 81-100: Very Elderly 
# 100+ Centenarian

def getlabel(age):
    if age <11:
        label = '1-10'
    elif age <21:
        label = '11-20'
    elif age <31:
        label = '21-30'
    elif age <41:
        label = '31-40'
    elif age <51:
        label = '41-50'
    elif age <71:
        label = '51-70'
    elif age <81:
        label = '71-80'  
    else:
        label = '81+'
    return label
def split_path(file_list,type,cat):
    
    for o,i in enumerate(file_list):

        img_data =Image.open(i)
        img_mode =img_data.mode
        img_size =img_data.size
        if img_mode != 'L':
            img_data = img_data.convert('L')
        if img_size != (48,48):
            img_data = img_data.resize((48,48))
            
        new_path = os.path.join('.././data/age_data_convert/',type,cat,f'{str(o)}.jpg')
        check_path = os.path.join('.././data/age_data_convert/',type,cat)
        if not(os.path.exists(check_path)):
            os.makedirs(check_path)
        img_data.save(new_path)
cat_dict = {}
cat_path = {}
for i in file_list:
    f_name = i.split(os.path.sep)[-1]
    age = int(f_name[:f_name.find('_')])
    label =getlabel(age)
    count=  cat_dict.get(label,1)
    list_cat=  cat_path.get(label,[])
    cat_dict[label] = count+1
    cat_path[label] = list_cat+ [i]
for cat in cat_path.keys():
    total_cat = len(cat_path[cat])
    train_cat = int(total_cat * 0.7)
    split_path(cat_path[cat][:train_cat],'train',cat)
    split_path(cat_path[cat][train_cat:],'test',cat)
    

In [ ]:
int(f_name[:f_name.find('_')])

In [ ]:
f_name

In [ ]:
cat_dict = {}
cat_path = {}
for i in file_list:
    i
  

In [ ]:
def caloutput(input,filter,stride,padding=0):
    # [(W−K+2P)/S]+1.
    # W is the input volume 
    # K is the Kernel size 
    # P is the padding 
    # S is the stride    
    return ((input-filter+2*padding)/stride)+1

In [ ]:
# 2024-1-1 train age model

In [ ]:
img_generator_train = ImageDataGenerator(rescale=1./255)

In [ ]:
path= '.././data/2023-12-28/ukdata/utkface_aligned_cropped/UTKFace'

In [ ]:
train_path = '.././data/age_data_convert/train'
gen_train = img_generator_train.flow_from_directory(train_path,batch_size=128,target_size=(48, 48),color_mode='grayscale')

test_path = '.././data/age_data_convert/test'
gen_test = img_generator_train.flow_from_directory(test_path,batch_size=128,target_size=(48, 48),color_mode='grayscale')

label = list(gen_train.class_indices.keys())

In [ ]:
# model
model = keras.Sequential()
model.add(layers.Conv2D(32,(3,3),padding="same",kernel_initializer='he_normal',activation='relu',input_shape=(48,48,1)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32,(3,3),padding="same",kernel_initializer='he_normal',activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(64,(3,3),padding="same",kernel_initializer='he_normal',activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64,(3,3),padding="same",kernel_initializer='he_normal',activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(label),activation='softmax'))


In [ ]:
los = keras.losses.CategoricalCrossentropy()
acc = keras.metrics.CategoricalAccuracy()
opt = optimizers.Adam(learning_rate=0.01)
epoch = 40
model.compile(optimizer=opt,loss=los,metrics=[acc])

In [ ]:
model.fit(gen_train,validation_data=gen_test,epochs=epoch)